In [31]:
from load_data import load_data
from preprocess import preprocess
from embed import Embedder
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import save_npz
import numpy as np

In [2]:
train_path_neg = "../twitter-datasets/train_neg.txt"
train_path_pos = "../twitter-datasets/train_pos.txt"
test_path = "../twitter-datasets/test_data.txt"

In [3]:
X_train, y_train, X_val, y_val, X_test = load_data(train_path_neg, train_path_pos, test_path, frac=0.2)

In [4]:
X_train_tokens, X_val_tokens, X_test_tokens = preprocess(X_train, X_val, X_test)

In [9]:
X_train_ = [' '.join(tokens) for tokens in X_train_tokens]
X_val_ = [' '.join(tokens) for tokens in X_val_tokens]
X_test_ = [' '.join(tokens) for tokens in X_test_tokens]

In [34]:
c_vec = CountVectorizer()
t_vec = TfidfVectorizer()
X_train_c_vec = c_vec.fit_transform(X_train_)
X_train_t_vec = t_vec.fit_transform(X_train_)
X_val_c_vec = c_vec.transform(X_val_)
X_test_c_vec = c_vec.transform(X_test_)
X_val_t_vec = t_vec.transform(X_val_)
X_test_t_vec = t_vec.transform(X_test_)

In [33]:
save_path = "../preprocessed_data/"
save_npz(save_path + 'X_train_c_vec', X_train_c_vec)
save_npz(save_path + 'X_train_t_vec', X_train_t_vec)
save_npz(save_path + 'X_val_c_vec', X_val_c_vec)
save_npz(save_path + 'X_test_c_vec', X_test_c_vec)
save_npz(save_path + 'X_val_t_vec', X_val_t_vec)
save_npz(save_path + 'X_test_t_vec', X_test_t_vec)
np.savez(save_path + "y_train", data=y_train.values)
np.savez(save_path + "y_val", data=y_val.values)

In [11]:
lr_c = LogisticRegression(max_iter=10000)
lr_c.fit(X_train_c_vec, y_train)

LogisticRegression(max_iter=10000)

In [12]:
lr_t = LogisticRegression(max_iter=10000)
lr_t.fit(X_train_t_vec, y_train)

LogisticRegression(max_iter=10000)

In [13]:
y_pred = lr_c.predict(X_val_c_vec)
print(accuracy_score(y_val, y_pred))

y_pred = lr_t.predict(X_val_t_vec)
print(accuracy_score(y_val, y_pred))

0.772
0.7685


In [14]:
sgdc_c = SGDClassifier()
sgdc_c.fit(X_train_c_vec, y_train)

SGDClassifier()

In [15]:
sgdc_t = SGDClassifier()
sgdc_t.fit(X_train_t_vec, y_train)

SGDClassifier()

In [16]:
y_pred = sgdc_c.predict(X_val_c_vec)
print(accuracy_score(y_val, y_pred))

y_pred = sgdc_t.predict(X_val_t_vec)
print(accuracy_score(y_val, y_pred))

0.764
0.764


In [17]:
rfc_c = RandomForestClassifier(verbose=2)
rfc_c.fit(X_train_c_vec, y_train)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  2.3min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

RandomForestClassifier(verbose=2)

In [16]:
rfc_t = RandomForestClassifier(verbose=1)
rfc_t.fit(X_train_t_vec, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.7min


RandomForestClassifier(verbose=1)

In [18]:
y_pred = rfc_c.predict(X_val_c_vec)
print(accuracy_score(y_val, y_pred))

y_pred = rfc_t.predict(X_val_t_vec)
print(accuracy_score(y_val, y_pred))

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.1s


0.753


NameError: name 'rfc_t' is not defined